In [114]:
!pip install pysrt tqdm pprint

  Using cached https://files.pythonhosted.org/packages/21/0a/47fdf541c97fd9b6a610cb5fd518175308a7cc60569962e776ac52420387/beautifulsoup4-4.6.3-py3-none-any.whl


In [47]:
!pip install -e git+https://github.com/yoyonel/py-googletrans.git#egg=googletrans

Obtaining googletrans from git+https://github.com/yoyonel/py-googletrans.git#egg=googletrans
  Updating /home/latty/.virtualenvs/py3.6_pydbsrt/src/googletrans clone
  Found existing installation: googletrans 2.3.0
    Uninstalling googletrans-2.3.0:
      Successfully uninstalled googletrans-2.3.0
  Running setup.py develop for googletrans


In [137]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [48]:
srt_fr_path = "The Golden Child.DVD_PAL.fre.srt"
srt_en_path = "The Golden Child (1986) 1080p web.en.srt"

In [49]:
import pysrt

srt_fr = pysrt.open(srt_fr_path)
srt_en = pysrt.open(srt_en_path)

In [50]:
srt_en[0].text, srt_fr[2].text

("Don't let him touch you!", 'Qu’il ne vous touche pas.')

In [51]:
srt_fr.shift(ratio=25.00/23.9)

In [52]:
diff_starts = srt_fr[2].start - srt_en[0].start
print(diff_starts)
#srt_fr.shift(seconds=diff_starts.seconds, milliseconds=diff_starts.milliseconds)

00:00:00,261


In [53]:
[
    (srt_fr[i+2].text, srt_en[i].text, srt_fr[i+2].start - srt_en[i].start)
    for i in range(100)
]

[('Qu’il ne vous touche pas.',
  "Don't let him touch you!",
  SubRipTime(0, 0, 0, 261)),
 ('<i>Tartes aux poils.</i>', '<i>Butt Pie.</i>', SubRipTime(0, 0, 0, 894)),
 ('<i>Gâteaux aux poils,</i>\nc’est la suite d’un bouquin.',
  "There's a sequel to a book written\ncalled <i>Butt Cake.</i>",
  SubRipTime(0, 0, 0, 846)),
 ('Ça marche bien dans les kiosques.\nUn cul plein de gâteau.',
  'Very popular at the news-stands.\nAbout a butt with cake all over it.',
  SubRipTime(0, 0, 0, 842)),
 ('<i>Gros culs</i>', '<i>Chunky Asses?</i>', SubRipTime(0, 0, 0, 950)),
 ('<i>Bienvenue au Journal Backman.\nJe suis Mel Backman.</i>',
  "<i>Welcome to the Mel Backman Journal.</i>\n<i>I'm Mel Backman.</i>",
  SubRipTime(0, 0, 1, 86)),
 ('<i>Je suis en compagnie\nde Chandler Jarrell...</i>',
  '<i>With me today is Chandler Jarrell</i>',
  SubRipTime(0, 0, 1, 123)),
 ('<i>dont l’action est très utile\nà la société.</i>',
  "<i>who's doing something</i>\n<i>very vital for the community.</i>",
  SubRipTim

In [54]:
from googletrans import Translator

translator = Translator()

In [55]:
from itertools import zip_longest
from tqdm import tqdm_notebook as tqdm

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [141]:
chunk_size = 15

# don't work ! :'( => not exactly ... 
# Seems google track IPs with "high" rate requests and bann (temporaly) IPs
for slice_srt_text in tqdm(
    map(lambda g: "#".join(g), 
        grouper(map(lambda srt: srt.text, srt_en), chunk_size, fillvalue="")),
    total=len(srt_en) // chunk_size
):
    try:
        t_slice_srt_text = translator.translate(slice_srt_text,
                                                src='en',
                                                dest='fr')
    except Exception as e:
        pass
    else:
        print(f"{t_slice_srt_text.text}")

- <i> Voici Teddy qui a 80 ans. </ i>
- <i> 80? </ i> # - <i> n'est-il pas chéri? </ i>
- <i> Teddy peut embrasser mon cul. </ i> # <i> J'essaie de trouver cette fille, </ i>
<I> Cheryl Mosely. </ I> # <i> Elle a été vue pour la dernière fois </ i>
<I> sur Vineland à North Hollywood. </ I> # <i> Elle avait un pull et une jupe. </ I>
<I> Appelez-moi au 4688492 si vous avez ... </ i> # <i> Tais-toi. Si vous avez des informations </ i>
<I> à propos d'elle, s'il vous plaît contactez-moi. </ i> # <i> Si vous ne remettez pas l'appareil photo, </ i>
<I> Je vais te casser le cul. Laisse le sur moi. </ I> # <i> Cheryl Mosely, Chandler Jarrell, </ i>
<I> 24 heures par jour, 4688492. </ i> # <i> Maintenant, vous pouvez parler avec les tortues. </ I>
<I> Excusez-moi. </ I> # <i> Merci. C'était Chandler Jarrell. </ I> # <i> Vous avez changé de sexe? </ I> #OK, allons-y. Prenez le lent, maintenant.
Attachez-le, attachez-le. # Pourquoi vous pendez
à l'arrière du terrain? # Allons-y. Voilà.
Tout le ch

Non, je suis l'agent Jarrell de l'américain
Stolen Artefacts Foundation. # Tout le monde va bien! je suis avec
la Fondation des artefacts volés! #Freddie, sors de ma vue!
Tu me rends malade! # Je te parlerai dans l'avion.
Personne ne soit alarmé! # Je suis Chandler Jarrell,
American Stolen Artefacts Foundation. # Grâce à vous, nous avons pu
pour attraper cet homme, Freddie, # le morceau de merde là-bas. # Je remercie les braves gens du Népal. # Vous êtes tous brillants. Merci, monsieur. # Vous êtes un enfer de leader.
La ruse ... l'instinct ... # L'instinct est brillant. Je peux dire
que vous avez formé sous cet homme. # Vous avez le même éclat de brillance
dans tes yeux. # Tu ne savais pas ce que tu faisais,
mais vous êtes brillant. # Le meilleur brillant est né
de quelqu'un d'ignorant ... # Oublie ça.
Voici mon assistante, Kee Nang.
Nous ne voudrions pas cela. Écoute ... # Amène-moi le garçon
et je te donnerai le couteau. # Ensuite, personne n’aura
être détruit "comme ça" # Réfléchis

In [142]:
chunk_size = 15

# don't work ! :'( => not exactly ... 
# Seems google track IPs with "high" rate requests and bann (temporaly) IPs
for grp_srt_text in tqdm(grouper(map(lambda srt: srt.text, srt_en), chunk_size, fillvalue=""),
                           total=len(srt_en) // chunk_size):
    try:
        t_slice_srt_text = translator.translate(list(grp_srt_text), src='en', dest='fr')
    except Exception as e:
        pass
    else:
        pp.pprint(f"{list(map(lambda s: s.text, t_slice_srt_text))}")

KeyboardInterrupt: 

In [58]:
import urllib.request
from urllib.parse import quote
import sys

typ = sys.getfilesystemencoding()

def translate(querystr, to_l="zh", from_l="en"):
    '''for google tranlate by doom
    '''
    C_agent = {'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.165063 Safari/537.36 AppEngine-Google."}
    flag = 'class="t0">'
    tarurl = ("http://translate.google.cn/m?hl=%s&sl=%s&q=%s" % (to_l, from_l, quote(querystr, safe='')))
    request = urllib.request.Request(tarurl, headers=C_agent)
    page = str(urllib.request.urlopen(request).read().decode(typ))
    target = page[page.find(flag) + len(flag):]
    target = target.split("<")[0]
    return target

In [173]:
from urllib.parse import unquote
from html import unescape  # https://docs.python.org/3/library/html.html
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

cleanr = re.compile('<.*?>')

# big chunk
# TODO: need to find the maximum size allowed to transfer
chunk_size = 50
split_token = ' # '

for slice_srt_text in tqdm(
        map(lambda g: split_token.join(
            map(lambda s: ''.join(cleanhtml(s)), 
                filter(lambda s: s is not None, g))),
            grouper(map(lambda srt: srt.text, srt_en), chunk_size, fillvalue=None)),
    total=len(srt_en) // chunk_size
):
    try:
        t_slice_srt_text = unescape(translate(slice_srt_text, from_l='en', to_l='fr'))
    except Exception as e:
        break
    else:
        # https://stackoverflow.com/questions/44780357/how-to-use-newline-n-in-f-string-to-format-output-in-python-3-6
#         t_srt_text = "\n¶ ".join(t_slice_srt_text.split('¶'))
#         print(f"{t_srt_text}")
        pp.pprint(list(zip(slice_srt_text.split(split_token), t_slice_srt_text.split(split_token))))
        print(f"{len(slice_srt_text.split(split_token))} - {len(t_slice_srt_text.split(split_token))}")


[   ("Don't let him touch you!", 'Ne le laisse pas te toucher!'),
    ('Butt Pie.', 'Butt Pie.'),
    (   "There's a sequel to a book written\ncalled Butt Cake.",
        'Il y a une suite à un livre écrit appelé Butt Cake.'),
    (   'Very popular at the news-stands.\nAbout a butt with cake all over it.',
        "Très populaire aux kiosques à journaux. À propos d'un mégot avec un "
        'gâteau partout.'),
    ('Chunky Asses?', 'Culs Chunky?'),
    (   "Welcome to the Mel Backman Journal.\nI'm Mel Backman.",
        'Bienvenue dans le Mel Backman Journal. Je suis Mel Backman. Chandler '
        'Jarrell'),
    (   'With me today is Chandler Jarrell',
        'qui fait quelque chose avec moi très vital pour la communauté.'),
    (   "who's doing something\nvery vital for the community.",
        "Chandler est un chercheur d'enfants perdus. Ai-je raison?"),
    (   'Chandler is a finder of lost children.\nAm I correct?',
        "- Oui, je trouve des enfants disparus. - C'est très i

[   ('...the Golden Child. Exactly.', "... l'enfant d'or. Exactement."),
    ('Right.', 'Droite.'),
    (   "- But why would they kill Cheryl?\n- I don't know.",
        '- Mais pourquoi tueraient-ils Cheryl? - Je ne sais pas.'),
    (   '- Why are they feeding the kid blood?\n- Are you sure?',
        "- Pourquoi nourrissent-ils le sang de l'enfant? - Êtes-vous sûr?"),
    (   '- I found a bowl of oatmeal...\n- Yes?',
        "- J'ai trouvé un bol de flocons d'avoine ... - Oui?"),
    (   "...with some blood on the bottom of it.\nIt's probably hers.",
        "... avec du sang sur le fond. C'est probablement le sien."),
    (   "I don't know. There is somebody\nwe could ask about the blood.",
        "Je ne sais pas. Il y a quelqu'un nous pourrions poser des questions "
        'sur le sang.'),
    (   "This is LA, though. You're going to\nask somebody about the blood?",
        "Ceci est LA, cependant. Vous allez demander à quelqu'un à propos du "
        'sang?'),
    (   'Are they 

[   (   "- I've never seen her before.\n- OK.",
        "- Je ne l'ai jamais vue auparavant. - D'ACCORD."),
    ('Oh, Cheryl!', 'Oh, Cheryl!'),
    ('Yeah, her... We sold her.', "Ouais, elle ... On l'a vendue."),
    (   'What do you mean?\nYou sold her to who?',
        "Que voulez-vous dire? Vous l'avez vendue à qui?"),
    (   "To Tommy Tong. He's got that\nrestaurant down on Broadway.",
        'À Tommy Tong. Il a ça restaurant sur Broadway.'),
    (   'We traded her for a case of cigarettes\n'
        'and a quart of pork fried rice.',
        "Nous l'avons échangée contre une caisse de cigarettes et une pinte de "
        'riz frit au porc.'),
    (   'Why would somebody\nwant to buy somebody?',
        "Pourquoi quelqu'un voudrait veux acheter quelqu'un?"),
    (   '- He needed a girl.\n- For what?',
        "- Il avait besoin d'une fille. - Pour quoi?"),
    (   'He worked out a deal with the Devil.\nHe said he needed her blood.',
        "Il a conclu un accord avec le diable. 

[   ('His death was a great loss.', 'Sa mort fut une grande perte.'),
    (   'Sardo needs it to kill the child,\n'
        'but you can only use it to save him.',
        "Sardo en a besoin pour tuer l'enfant, mais vous ne pouvez l'utiliser "
        'que pour le sauver.'),
    (   'You must obtain the knife and\nlure Numspa into freeing the child.',
        "Vous devez obtenir le couteau et inciter Numspa à libérer l'enfant."),
    (   'You must never let him\nget possession of the knife.',
        'Vous ne devez jamais le laisser prendre possession du couteau.'),
    (   'That sounds like a neat trick.\nHow can I pull it off?',
        'Cela ressemble à une astuce. Comment puis-je le retirer?'),
    (   'If need be, we will exchange\nthe dagger for the child.',
        "Au besoin, nous échangerons le poignard pour l'enfant."),
    (   'If the Abbot of Karma Tang\nwill let us have it.',
        "Si l'abbé de Karma Tang nous laisserons l'avoir."),
    (   "If the kid's so important, w

[   (   'I humbly beg you,\nlet us have the knife.',
        'Je vous en prie humblement laissez-nous le couteau.'),
    ('Let him ask it.', 'Laisse-le lui demander.'),
    ('I-I-I... want the knife.', 'Je-je-je ... veux le couteau.'),
    ('Let him ask again.', 'Laissez-le demander à nouveau.'),
    ('I want the knife.', 'Je veux le couteau.'),
    ('Please.', "S'il vous plaît."),
    (   'Only a man whose heart is pure\ncan wield the knife.',
        'Seulement un homme dont le coeur est pur peut brandir le couteau.'),
    (   "If you're such a man,\nyou will have it.",
        "Si tu es un tel homme, vous l'aurez."),
    (   '"Only a man whose heart is pure\ncan wield the knife."',
        '"Seulement un homme dont le coeur est pur peut brandir le couteau. "# '
        'Et seulement un homme dont le cul est étroit peut descendre ces '
        'étapes.'),
    (   'And only a man whose ass is narrow\ncan get down these steps.',
        "Et si le mien est un tel âne, alors je l'aurai."

[   (   '- Come on.\n- You trust in your destiny.',
        '- Allons. - Vous avez confiance en votre destin.'),
    (   '- Come with me.\n- I have nothing to declare.',
        "- Viens avec moi. - Je n'ai rien à déclarer."),
    (   '- But necessary.\n- Why? I can show you my bag here.',
        '- Mais nécessaire. - Pourquoi? Je peux te montrer mon sac ici.'),
    (   '- For your own protection.\n- These are just my drawers.',
        '- Pour votre propre protection. - Ce ne sont que mes tiroirs.'),
    (   'These are clean\nand those are dirty.',
        'Ils sont propres et ceux-là sont sales.'),
    (   'Excuse me, I have a plane to catch.\nCome on, hurry up.',
        "Excusez-moi, j'ai un avion à prendre. Allez dépêche toi."),
    ("- What's this?\n- Nothing.", "- Qu'est-ce que c'est ça? - Rien."),
    (   'Just once I want to go back to America.\n'
        "I'm a citizen. What are you looking at?",
        "Juste une fois, je veux retourner en Amérique. Je suis citoyen Qu'est 

[   (   "You don't want to go to Tibet\nwith a big... on your head.",
        'Vous ne voulez pas aller au Tibet avec un gros ... sur la tête.'),
    (   'That is a cool trick. Did you ever\n'
        'think about going into show business?',
        "C'est un truc cool. Avez-vous déjà penser à aller dans le show "
        'business?'),
    (   "I could be your partner,\nI'd throw a rock at you,",
        'Je pourrais être votre partenaire, Je te jetterais une pierre'),
    (   "and you'd make it float off\nand hit the wall. Standing ovations.",
        'et tu la ferais flotter et a frappé le mur. Ovations.'),
    (   "Do they have\nEd MacMahon's Starsearch in Tibet?",
        "Ont-ils La recherche d'étoiles d'Ed MacMahon au Tibet?"),
    (   'Probably not.\nThey probably have Foodsearch.',
        'Probablement pas. Ils ont probablement Foodsearch.'),
    (   'We could go on Starsearch and\nlet the audience throw rocks at you.',
        'Nous pourrions continuer sur Starsearch et laiss